В формулировке заданий будет использоваться понятие **worker**. Это слово обозначает какую-то единицу параллельного выполнения, в случае питона это может быть **поток** или **процесс**, выбирайте то, что лучше будет подходить к конкретной задаче

В каждом задании нужно писать подробные аннотиции типов для:
1. Аргументов функций и классов
2. Возвращаемых значений
3. Классовых атрибутов (если такие есть)

В каждом задании нужно писать докстроки в определённом стиле (какой вам больше нравится) для всех функций, классов и методов

# Задание 1 (7 баллов)

В одном из заданий по ML от вас требовалось написать кастомную реализацию Random Forest. Её проблема состоит в том, что она работает медленно, так как использует всего один поток для работы. Добавление параллельного программирования в код позволит получить существенный прирост в скорости обучения и предсказаний.

В данном задании от вас требуется добавить возможность обучать случайный лес параллельно и использовать параллелизм для предсказаний. Для этого вам понадобится:
1. Добавить аргумент `n_jobs` в метод `fit`. `n_jobs` показывает количество worker'ов, используемых для распараллеливания
2. Добавить аргумент `n_jobs` в методы `predict` и `predict_proba`
3. Реализовать функционал по распараллеливанию в данных методах

В результате код `random_forest.fit(X, y, n_jobs=2)` и `random_forest.predict(X, y, n_jobs=2)` должен работать в ~1.5-2 раза быстрее, чем `random_forest.fit(X, y, n_jobs=1)` и `random_forest.predict(X, y, n_jobs=1)` соответственно

Если у вас по каким-то причинам нет кода случайного леса из ДЗ по ML, то вы можете написать его заново или попросить у однокурсника. *Детали* реализации ML части оцениваться не будут, НО, если вы поломаете логику работы алгоритма во время реализации параллелизма, то за это будут сниматься баллы

В задании можно использовать только модули из **стандартной библиотеки** питона, а также функции и классы из **sklearn** при помощи которых вы изначально писали лес

In [15]:
from sklearn.base import BaseEstimator
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp
import numpy as np
from typing import Callable, Any, Optional, Union
SEED = 1999

In [10]:
# # Случайный лес обычный
# def fit_wj(forest, X, y):
#     forest.classes_ = sorted(np.unique(y))

#     if forest.max_features > X.shape[1] and forest.max_features != None:
#         forest.max_features = X.shape[1]
#         print("Your max_features is greater then exicted features in dataset.\n \
#         Maximal possible number of features assigned to max_features")


#     for i in range(forest.n_estimators):
#         rando = np.random.seed(forest.random_state + i)
#         forest.feat_ids_by_tree.append(np.sort(np.random.choice(X.shape[1], size=forest.max_features,
#                                                 replace=False)))
#         pseudosample = np.random.choice(X.shape[0], size=X.shape[0], 
#                              replace=True)

#         x_pre_bts = X[pseudosample]
#         x_bts = x_pre_bts[:, forest.feat_ids_by_tree[i]]
#         y_bts = y[pseudosample]            

#         dtree = DecisionTreeClassifier(max_depth=forest.max_depth, 
#                                        max_features=forest.max_features, 
#                                        random_state=rando)

#         forest.trees.append(dtree.fit(x_bts, y_bts))

#     return forest


# class RandomForestClassifierCustom(BaseEstimator):
#     def __init__(
#         self, n_estimators=10, max_depth=None, max_features=None, random_state=SEED
#     ):
#         self.n_estimators = n_estimators
#         self.max_depth = max_depth
#         self.max_features = max_features
#         self.random_state = random_state

#         self.trees = []
#         self.feat_ids_by_tree = []


#     def fit(self, X, y, n_jobs=1):
#         with ProcessPoolExecutor(n_jobs) as pool:
#             results = list(pool.map(fit_wj, self, X, y))
# #         self.classes_ = sorted(np.unique(y))
        
# #         if self.max_features > X.shape[1] and self.max_features != None:
# #             self.max_features = X.shape[1]
# #             print("Your max_features is greater then exicted features in dataset.\n \
# #             Maximal possible number of features assigned to max_features")
            
        
# #         for i in range(self.n_estimators):
# #             rando = np.random.seed(self.random_state + i)
# #             self.feat_ids_by_tree.append(np.sort(np.random.choice(X.shape[1], size=self.max_features,
# #                                                     replace=False)))
# #             pseudosample = np.random.choice(X.shape[0], size=X.shape[0], 
# #                                  replace=True)
            
# #             x_pre_bts = X[pseudosample]
# #             x_bts = x_pre_bts[:, self.feat_ids_by_tree[i]]
# #             y_bts = y[pseudosample]            
            
# #             dtree = DecisionTreeClassifier(max_depth=self.max_depth, 
# #                                            max_features=self.max_features, 
# #                                            random_state=rando)
            
# #             self.trees.append(dtree.fit(x_bts, y_bts))
        
# #         return self
            

#     def predict_proba(self, X):
#         preds = []
        
#         for i in range(self.n_estimators):
#             preds.append(self.trees[i].predict_proba(X[:, self.feat_ids_by_tree[i]]))
        
#         return np.mean(preds, axis=0)
    
#     def predict(self, X):
#         probas = self.predict_proba(X)
#         predictions = np.argmax(probas, axis=1)
        
#         return predictions

In [7]:
# # # Случайный лес с процессами

# # class Forester_ripper(mp.Process):
# #     def __init__(self, Sself, X, y, range_custom):
# #         super().__init__()
# #         self.Sself = Sself
# #         self.X = X
# #         self.y = y
# #         self.range_custom = range_custom
    
# #     def run(self):
# #         result_self = fit_wj(self.Sself, self.X, self.y, self.range_custom)
        
            
            
            
# class RandomForestClassifierCustom(BaseEstimator):
#     def __init__(
#         self, n_estimators=10, max_depth=None, max_features=None, random_state=SEED
#     ):
#         self.n_estimators = n_estimators
#         self.max_depth = max_depth
#         self.max_features = max_features
#         self.random_state = random_state

#         self.trees = []
#         self.feat_ids_by_tree = []

#     def fit(self, X, y, n_jobs=1):
#         self.X = X
#         self.y = y
# #         numbers = self.n_estimators
# #         add_const = numbers % n_jobs
# #         if add_const == 0:
# #             processes = []
# #             for job in range(n_jobs):
# #                 custom_range = range(0 + (numbers // n_jobs) * job, (numbers // n_jobs) * (job + 1))
# #                 processes.append(Forester_ripper(self, X, y, custom_range))
# #         else:
# #             processes = []
# #             for job in range(n_jobs - 1):
# #                 custom_range = range(0 + (numbers // n_jobs) * job, (numbers // n_jobs) * (job + 1))
# #                 processes.append(Forester_ripper(self, X, y, custom_range))
# #             custom_range = range(numbers - (numbers // n_jobs + add_const), numbers)
# #             processes.append(Forester_ripper(self, X, y, custom_range))
            
# #         for process in processes:
# #             process.start()
# #         for process in processes:
# #             process.join()  

        
#     def fit_wj(self, number):
#         self.classes_ = sorted(np.unique(y))
#         number2 = 0
        
#         if self.max_features > self.X.shape[1] and self.max_features != None:
#             self.max_features = self.X.shape[1]
#             print("Your max_features is greater then exicted features in dataset.\n \
#             Maximal possible number of features assigned to max_features")
            
        
#         for number in range_custom
#             number2 = number - range_custom[0]
#             rando = np.random.seed(self.random_state + number)
#             self.feat_ids_by_tree.append(np.sort(np.random.choice(self.XX.shape[1], size=self.max_features,
#                                                     replace=False)))
#             pseudosample = np.random.choice(X.shape[0], size=self.X.shape[0], 
#                                  replace=True)

#             x_pre_bts = X[pseudosample]
#             x_bts = x_pre_bts[:, self.feat_ids_by_tree[number2]]
#             y_bts = y[pseudosample]            

#             dtree = DecisionTreeClassifier(max_depth=self.max_depth, 
#                                            max_features=self.max_features, 
#                                            random_state=rando)

#             self.trees.append(dtree.fit(x_bts, y_bts))
#             number2 = 0

#         return self
            

#     def predict_proba(self, X, n_jobs=1):
#         preds = []
        
#         for i in range(self.n_estimators):
#             preds.append(self.trees[i].predict_proba(X[:, self.feat_ids_by_tree[i]]))
        
#         return np.mean(preds, axis=0)
    
#     def predict(self, X, n_jobs=1):
#         probas = self.predict_proba(X)
#         predictions = np.argmax(probas, axis=1)
        
#         return predictions
    

In [11]:
# X, y = make_classification(n_samples=100000)
# random_forest = RandomForestClassifierCustom(max_depth=30, n_estimators=10, max_features=2, random_state=42)

In [12]:
# %%time

# _ = random_forest.fit(X, y)

TypeError: 'RandomForestClassifierCustom' object is not iterable

In [19]:
%%time

preds = random_forest.predict(X)

CPU times: user 155 ms, sys: 0 ns, total: 155 ms
Wall time: 153 ms


In [20]:
preds

array([0, 1, 1, ..., 1, 1, 1])

In [64]:
%%time

_ = random_forest.fit(X, y, n_jobs=1)

CPU times: user 372 µs, sys: 7.75 ms, total: 8.13 ms
Wall time: 15.8 ms


In [65]:
%%time

preds_1 = random_forest.predict(X, n_jobs=1)

IndexError: list index out of range

In [279]:
random_forest = RandomForestClassifierCustom(max_depth=30, n_estimators=10, max_features=2, random_state=42)

In [280]:
%%time

_ = random_forest.fit(X, y, n_jobs=2)

CPU times: user 782 µs, sys: 84.4 ms, total: 85.2 ms
Wall time: 2.18 s


In [281]:
%%time

preds_2 = random_forest.predict(X, n_jobs=2)

CPU times: user 21 ms, sys: 75.4 ms, total: 96.4 ms
Wall time: 140 ms


In [284]:
(preds_1 == preds_2).all()   # Количество worker'ов не должно влиять на предсказания

True

#### Какие есть недостатки у вашей реализации параллельного Random Forest (если они есть)? Как это можно исправить? Опишите словами, можно без кода (+1 дополнительный балл)

1.
- Проблема: Я потратил на нее много времени и сильно подтормаживал (хотя это скорее недостатки моей реализации).
- Решение: Попрактиваться в многопоточном/процессовом программировании еще.
2.
- Проблема:
- Решение:

# Задание 2 (9 баллов)

Напишите декоратор `memory_limit`, который позволит ограничивать использование памяти декорируемой функцией.

Декоратор должен принимать следующие аргументы:
1. `soft_limit` - "мягкий" лимит использования памяти. При превышении функцией этого лимита должен будет отображён **warning**
2. `hard_limit` - "жёсткий" лимит использования памяти. При превышении функцией этого лимита должно будет брошено исключение, а функция должна немедленно завершить свою работу
3. `poll_interval` - интервал времени (в секундах) между проверками использования памяти

Требования:
1. Потребление функцией памяти должно отслеживаться **во время выполнения функции**, а не после её завершения
2. **warning** при превышении `soft_limit` должен отображаться один раз, даже если функция переходила через этот лимит несколько раз
3. Если задать `soft_limit` или `hard_limit` как `None`, то соответствующий лимит должен быть отключён
4. Лимиты должны передаваться и отображаться в формате `<number>X`, где `X` - символ, обозначающий порядок единицы измерения памяти ("B", "K", "M", "G", "T", ...)
5. В тексте warning'ов и исключений должен быть указан текщий объём используемой памяти и величина превышенного лимита

В задании можно использовать только модули из **стандартной библиотеки** питона, можно писать вспомогательные функции и/или классы

В коде ниже для вас предопределены некоторые полезные функции, вы можете ими пользоваться, а можете не пользоваться

In [23]:
import os
import sys
import signal
import psutil
import time
import threading
import warnings

       
class FuncInterrupter(threading.Thread):
    def __init__(self, before: int, soft: str, hard: str, time_gap: Union[float, int], func: Callable[Any, Any]) -> None:
        """
        Initalizes thread with function interrupter.

        :param before: bytes of memory in work.
        :type before: int
        :param soft: Soft memory limit for decorated function.
        :type soft: str
        :param hard: Hard memory limit for decorated function.
        :type hard: str
        :param time_gap: The time between two monitoring episodes.
        :type time_gap: int or float
        :param func: Function, which we have to checked.
        :type func: int or float
        :return: None. It is initializer.
        """
        super().__init__()
        self.before = before
        self.soft = human_readable_to_bytes(soft)
        self.hard = human_readable_to_bytes(hard)
        self.time_gap = time_gap
        self.func = func
        self.flag = "No"

    def run(self):
        """
        Runs thread with function interrupter. 
        If function extends soft limit, warning is raised.
        If function extends hard limit, exception mimicry is raised

        :return: None.
        """
        while True:
            time.sleep(self.time_gap)
            self.after = get_memory_usage()
            self.diff = self.after - self.before 
            if self.diff > self.hard:
                e_text = f"Your soft memory limit is {bytes_to_human_readable(self.soft)}\n\
                and your hard memory limit is {bytes_to_human_readable(self.hard)}.\n\
                Now you are using {bytes_to_human_readable(self.diff)}, which is on {bytes_to_human_readable(self.diff - self.hard)} bigger than your hard limit"
                print(e_text, file=sys.stderr)
                os._exit(1)    
            elif self.diff > self.soft and self.flag == "No":
                self.flag = "Yes"
                w_text = f"Your soft memory limit is {bytes_to_human_readable(self.soft)}\n\
                and your hard memory limit is {bytes_to_human_readable(self.hard)}.\n\
                Now you are using {bytes_to_human_readable(self.diff)}, which is on {bytes_to_human_readable(self.diff - self.soft)} bigger than your soft limit"
                warnings.warn(w_text) 
            time.sleep(self.time_gap)
        
            
def get_memory_usage() -> int:    # Показывает текущее потребление памяти процессом
    """
    Shows memory usage by process. 

    :return: bytes for current process
    :rtype: int
    """
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss


def bytes_to_human_readable(n_bytes: int) -> str:
    """
    Converts bytes to human-readable form. 
    
    :param n_bytes: number of bytes used for current process.
    :type before: int    
    :return: bytes in human-readable form
    :rtype: str
    """
    symbols = ('K', 'M', 'G', 'T', 'P', 'E', 'Z', 'Y')
    prefix = {}
    for idx, s in enumerate(symbols):
        prefix[s] = 1 << (idx + 1) * 10
    for s in reversed(symbols):
        if n_bytes >= prefix[s]:
            value = float(n_bytes) / prefix[s]
            return f"{value:.2f}{s}"
    return f"{n_bytes}B"


def human_readable_to_bytes(string: str) -> int:
    """
    Converts human-readable memory to bytes. 
    
    :param n_bytes: bytes in human-readable form.
    :type before: str   
    :return: number of bytes used for current process
    :rtype: int 
    """
    number = float(string[:-1])
    letter = string[-1] 
    letters_to_bytes = {
        'K':1024, 'M':1024**2, 'G':1024**3, 'T':1024**4, 'P':1024**5, 'E':1024**6, 'Z':1024**7, 'Y':1024**8
    }
    result = round(number * letters_to_bytes[letter], 2)
    return(result)


def memory_limit(soft_limit: Optional[str] = None, hard_limit: Optional[str] = None, poll_interval: int = 1) -> Callable[Callable[Any, Any], Callable[[Any, Any], None]]:
    """
    Decorator factory
    
    :param soft_limit: Soft memory limit for decorated function.
    :type soft_limit: str
    :param hard_limit:
    :type hard_limit: str
    :param poll_interval:
    :type poll_interval: int
    :return: decorator with limits and checker time gap
    :rtype: function
    """
    def real_decorator(func: Callable[Any, Any]) -> Callable[[Any, Any], None]:
        """
        Decorator

        :param func: function we need to change
        :type func: function
        :return: changed function
        :rtype: function
        """
        def obyortka(*args: Any, **kwargs: Any) -> None:
            """
            Wrapper function in decorator

            :param *args: arguments we need to put into the function
            :type func: function
            :param **kwargs: arguments we need to put into the function
            :type func: function
            :return: changed function
            :rtype: function
            """
            before = get_memory_usage()
            func_inter = FuncInterrupter(before, soft_limit, hard_limit, poll_interval, func)
            func_inter.start()
            func()
        return obyortka
    return real_decorator

In [24]:
@memory_limit(soft_limit="512M", hard_limit="1.5G", poll_interval=0.1)
def memory_increment() -> list[int]:
    """
    Функция для тестирования
    
    В течение нескольких секунд достигает использования памяти 1.89G
    Потребление памяти и скорость накопления можно варьировать, изменяя код
    
    :return: list of numbers, which are divided on 500000 without remainder
    :rtype: list of integers
    """
    lst = []
    for i in range(50000000):
        if i % 500000 == 0:
            time.sleep(0.1)
        lst.append(i)
    return lst

In [ ]:
memory_increment()

/tmp/ipykernel_34853/1000738977.py:36: UserWarning: Your soft memory limit is 512.00M
                and your hard memory limit is 1.50G.
                Now you are using 521.43M, which is on 9.43M bigger than your soft limit
  warnings.warn(w_text)


# Задание 3 (11 баллов)

Напишите функцию `parallel_map`. Это должна быть **универсальная** функция для распараллеливания, которая эффективно работает в любых условиях.

Функция должна принимать следующие аргументы:
1. `target_func` - целевая функция (обязательный аргумент)
2. `args_container` - контейнер с позиционными аргументами для `target_func` (по-умолчанию `None` - позиционные аргументы не передаются)
3. `kwargs_container` - контейнер с именованными аргументами для `target_func` (по-умолчанию `None` - именованные аргументы не передаются)
4. `n_jobs` - количество workers, которые будут использованы для выполнения (по-умолчанию `None` - количество логических ядер CPU в системе)

Функция должна работать аналогично `***PoolExecutor.map`, применяя функцию к переданному набору аргументов, но с некоторыми дополнениями и улучшениями
    
Поскольку мы пишем **универсальную** функцию, то нам нужно будет выполнить ряд требований, чтобы она могла логично и эффективно работать в большинстве ситуаций

1. `target_func` может принимать аргументы любого вида в любом количестве
2. Любые типы данных в `args_container`, кроме `tuple`, передаются в `target_func` как единственный позиционный аргумент. `tuple` распаковываются в несколько аргументов
3. Количество элементов в `args_container` должно совпадать с количеством элементов в `kwargs_container` и наоборот, также значение одного из них или обоих может быть равно `None`, в иных случаях должна кидаться ошибка (оба аргумента переданы, но размеры не совпадают)

4. Функция должна выполнять определённое количество параллельных вызовов `target_func`, это количество зависит от числа переданных аргументов и значения `n_jobs`. Сценарии могут быть следующие
    + `args_container=None`, `kwargs_container=None`, `n_jobs=None`. В таком случае функция `target_func` выполнится параллельно столько раз, сколько на вашем устройстве логических ядер CPU
    + `args_container=None`, `kwargs_container=None`, `n_jobs=5`. В таком случае функция `target_func` выполнится параллельно **5** раз
    + `args_container=[1, 2, 3]`, `kwargs_container=None`, `n_jobs=5`. В таком случае функция `target_func` выполнится параллельно **3** раза, несмотря на то, что `n_jobs=5` (так как есть всего 3 набора аргументов для которых нам нужно получить результат, а лишние worker'ы создавать не имеет смысла)
    + `args_container=None`, `kwargs_container=[{"s": 1}, {"s": 2}, {"s": 3}]`, `n_jobs=5`. Данный случай аналогичен предыдущему, но здесь мы используем именованные аргументы
    + `args_container=[1, 2, 3]`, `kwargs_container=[{"s": 1}, {"s": 2}, {"s": 3}]`, `n_jobs=5`. Данный случай аналогичен предыдущему, но здесь мы используем и позиционные, и именованные аргументы
    + `args_container=[1, 2, 3, 4]`, `kwargs_container=None`, `n_jobs=2`. В таком случае в каждый момент времени параллельно будет выполняться **не более 2** функций `target_func`, так как нам нужно выполнить её 4 раза, но у нас есть только 2 worker'а.
    + В подобных случаях (из примера выше) должно оптимизироваться время выполнения. Если эти 4 вызова выполняются за 5, 1, 2 и 1 секунды, то параллельное выполнение с `n_jobs=2` должно занять **5 секунд** (не 7 и тем более не 10)

5. `parallel_map` возвращает результаты выполнения `target_func` **в том же порядке**, в котором были переданы соответствующие аргументы
6. Работает с функциями, созданными внутри других функций

Для базового решения от вас не ожидается **сверххорошая** оптимизация по времени и памяти для всех возможных случаев. Однако за хорошо оптимизированную логику работы можно получить до **+3 дополнительных баллов**

Вы можете сделать класс вместо функции, если вам удобнее

В задании можно использовать только модули из **стандартной библиотеки** питона

Ниже приведены тестовые примеры по каждому из требований

In [25]:
import multiprocessing as mp
import concurrent.futures as cf

def parallel_map(target_func,
                 args_container=None,
                 kwargs_container=None,
                 n_jobs=None):
    
    if args_container is not None and kwargs_container is not None:
        if len(args_container) != len(kwargs_container):
            raise(Exception)
    
#     fp = 0
#     sp = 0
#     tp = 0
#     if args_container is not None:
#         fp = len(args_container)
#     if kwargs_container is not None:
#         sp = len(kwargs_container)
#     if n_jobs is not None:
#         tp = len(n_jobs)
        if n_jobs is None:
            num_procs = min(len(args_container), len(kwargs_container))
        else:
            num_procs = min(len(args_container), len(kwargs_container), n_jobs)
    
    if n_jobs is not None and args_container is None:
        if kwargs_container is not None:
            num_procs = min(len(kwargs_container), n_jobs)
        else:
            num_procs = n_jobs
    elif n_jobs is not None and kwargs_container is None and args_container is not None: 
        num_procs = min(len(args_container), n_jobs)
    elif n_jobs is None and args_container is None:
        if kwargs_container is not None:
            num_procs = len(kwargs_container)
        else:
            num_procs = None
    elif n_jobs is None and kwargs_container is None and args_container is not None: 
        num_procs = len(args_container)

        
        
    pe = cf.ProcessPoolExecutor(max_workers=num_procs)
    
    if args_container is not None and kwargs_container is not None:
        if type(args_container[0]) == tuple:
            result = list(pe.map(target_func, *args_container, **kwargs_container))
        else:
            result = list(pe.map(target_func, *args_container, **kwargs_container))
    elif args_container is not None and kwargs_container is None:
        if type(args_container) == tuple:
            result = list(pe.map(target_func, *args_container))
        else:
            result = list(pe.map(target_func, args_container))
    elif args_container is None and kwargs_container is None:
        result = list(pe.map(target_func))
    else:
        result = list(pe.map(target_func, **kwargs_container))
        
    return result


In [22]:
import time


# Это только один пример тестовой функции, ваша parallel_map должна уметь эффективно работать с ЛЮБЫМИ функциями
# Поэтому обязательно протестируйте код на чём-нибудбь ещё
def test_func(x=1, s=2, a=1, b=1, c=1):
    time.sleep(s)
    return a*x**2 + b*x + c

In [26]:
%%time

# Пример 2.1
# Отдельные значения в args_container передаются в качестве позиционных аргументов
parallel_map(test_func, args_container=[1, 2.0, 3j-1, 4])   # Здесь происходят параллельные вызовы: test_func(1) test_func(2.0) test_func(3j-1) test_func(4)

CPU times: user 973 µs, sys: 17.1 ms, total: 18.1 ms
Wall time: 2.02 s


[3, 7.0, (-8-3j), 21]

In [27]:
%%time

# Пример 2.2
# Элементы типа tuple в args_container распаковываются в качестве позиционных аргументов
parallel_map(test_func, [(1, 1), (2.0, 2), (3j-1, 3), 4])    # Здесь происходят параллельные вызовы: test_func(1, 1) test_func(2.0, 2) test_func(3j-1, 3) test_func(4)

TypeError: unsupported operand type(s) for ** or pow(): 'tuple' and 'int'

In [28]:
%%time

# Пример 3.1
# Возможна одновременная передача args_container и kwargs_container, но количества элементов в них должны быть равны
parallel_map(test_func,
             args_container=[1, 2, 3, 4],
             kwargs_container=[{"s": 3}, {"s": 3}, {"s": 3}, {"s": 3}])

# Здесь происходят параллельные вызовы: test_func(1, s=3) test_func(2, s=3) test_func(3, s=3) test_func(4, s=3)

TypeError: concurrent.futures.process.ProcessPoolExecutor.map() argument after ** must be a mapping, not list

In [29]:
%%time

# Пример 3.2
# args_container может быть None, а kwargs_container задан явно
parallel_map(test_func,
             kwargs_container=[{"s": 3}, {"s": 3}, {"s": 3}, {"s": 3}])

TypeError: concurrent.futures.process.ProcessPoolExecutor.map() argument after ** must be a mapping, not list

In [43]:
%%time

# Пример 3.3
# kwargs_container может быть None, а args_container задан явно
parallel_map(test_func,
             args_container=[1, 2, 3, 4])

CPU times: user 4.11 ms, sys: 9.2 ms, total: 13.3 ms
Wall time: 2.01 s


[3, 7, 13, 21]

In [30]:
%%time

# Пример 3.4
# И kwargs_container, и args_container могут быть не заданы
parallel_map(test_func)

CPU times: user 1.15 ms, sys: 0 ns, total: 1.15 ms
Wall time: 1.03 ms


[]

In [31]:
%%time

# Пример 3.4
# И kwargs_container, и args_container могут быть не заданы
parallel_map(test_func)

CPU times: user 1.09 ms, sys: 0 ns, total: 1.09 ms
Wall time: 835 µs


[]

In [32]:
%%time

# Пример 3.5
# При несовпадении количеств позиционных и именованных аргументов кидается ошибка
parallel_map(test_func,
             args_container=[1, 2, 3, 4],
             kwargs_container=[{"s": 3}, {"s": 3}, {"s": 3}])

Exception: 

In [33]:
%%time

# Пример 4.1
# Если функция не имеет обязательных аргументов и аргумент n_jobs не был передан, то она выполняется параллельно столько раз, сколько ваш CPU имеет логических ядер
# В моём случае это 24, у вас может быть больше или меньше
parallel_map(test_func)

CPU times: user 1.03 ms, sys: 0 ns, total: 1.03 ms
Wall time: 914 µs


[]

In [42]:
%%time

# Пример 4.2
# Если функция не имеет обязательных аргументов и передан только аргумент n_jobs, то она выполняется параллельно n_jobs раз
parallel_map(test_func, n_jobs=2)

CPU times: user 1.49 ms, sys: 0 ns, total: 1.49 ms
Wall time: 1.34 ms


[]

In [41]:
%%time

# Пример 4.3
# Если аргументов для target_func указано МЕНЬШЕ, чем n_jobs, то используется такое же количество worker'ов, сколько было передано аргументов
parallel_map(test_func,
             args_container=[1, 2, 3],
             n_jobs=5)   # Здесь используется 3 worker'a

CPU times: user 5.34 ms, sys: 8.02 ms, total: 13.4 ms
Wall time: 2.03 s


[3, 7, 13]

In [40]:
%%time

# Пример 4.4
# Аналогичный предыдущему случай, но с именованными аргументами
parallel_map(test_func,
             kwargs_container=[{"s": 3}, {"s": 3}, {"s": 3}],
             n_jobs=5)   # Здесь используется 3 worker'a

TypeError: concurrent.futures.process.ProcessPoolExecutor.map() argument after ** must be a mapping, not list

In [39]:
%%time

# Пример 4.5
# Комбинация примеров 4.3 и 4.4 (переданы и позиционные и именованные аргументы)
parallel_map(test_func,
             args_container=[1, 2, 3],
             kwargs_container=[{"s": 3}, {"s": 3}, {"s": 3}],
             n_jobs=5)   # Здесь используется 3 worker'a

TypeError: concurrent.futures.process.ProcessPoolExecutor.map() argument after ** must be a mapping, not list

In [38]:
%%time

# Пример 4.6
# Если аргументов для target_func указано БОЛЬШЕ, чем n_jobs, то используется n_jobs worker'ов
parallel_map(test_func,
             args_container=[1, 2, 3, 4],
             kwargs_container=None,
             n_jobs=2)   # Здесь используется 2 worker'a

CPU times: user 7.84 ms, sys: 4.31 ms, total: 12.2 ms
Wall time: 4.01 s


[3, 7, 13, 21]

In [37]:
%%time

# Пример 4.7
# Время выполнения оптимизируется, данный код должен отрабатывать за 5 секунд
parallel_map(test_func,
             kwargs_container=[{"s": 5}, {"s": 1}, {"s": 2}, {"s": 1}],
             n_jobs=2)

TypeError: concurrent.futures.process.ProcessPoolExecutor.map() argument after ** must be a mapping, not list

In [36]:
def test_func2(string, sleep_time=1):
    time.sleep(sleep_time)
    return string

# Пример 5
# Результаты возвращаются в том же порядке, в котором были переданы соответствующие аргументы вне зависимости от того, когда завершился worker
arguments = ["first", "second", "third", "fourth", "fifth"]
parallel_map(test_func2,
             args_container=arguments,
             kwargs_container=[{"sleep_time": 5}, {"sleep_time": 4}, {"sleep_time": 3}, {"sleep_time": 2}, {"sleep_time": 1}])

TypeError: concurrent.futures.process.ProcessPoolExecutor.map() argument after ** must be a mapping, not list

In [35]:
%%time


def test_func3():
    def inner_test_func(sleep_time):
        time.sleep(sleep_time)
    return parallel_map(inner_test_func, args_container=[1, 2, 3])

# Пример 6
# Работает с функциями, созданными внутри других функций
test_func3()

AttributeError: Can't pickle local object 'test_func3.<locals>.inner_test_func'